In [1]:
import tensorflow as tf
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "not available")


GPU is available


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow_datasets as tfds

# 1. Carregar o dataset "cats_vs_dogs"
(train_ds, test_ds) = tfds.load(
    "cats_vs_dogs",
    split=["train[:80%]", "train[80%:]"],  # 80% para treino, 20% para teste
    as_supervised=True,  # Retorna pares (imagem, rótulo)
)

# 2. Pré-processamento das imagens
def preprocess(image, label):
    image = tf.image.resize(image, (150, 150))  # Redimensionar as imagens para 150x150
    image = preprocess_input(image)            # Pré-processar para VGG16
    label = tf.one_hot(label, depth=2)         # Transformar os rótulos para one-hot encoding
    return image, label

train_ds = train_ds.map(preprocess).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

# 3. Carregar o modelo base VGG16 pré-treinado no ImageNet
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False  # Congelar as camadas do modelo base

# 4. Criar o modelo com camadas adicionais para classificação
model = models.Sequential([
    base_model,                               # Modelo VGG16 como extrator de características
    layers.Flatten(),                         # Achatar as features extraídas
    layers.Dense(256, activation="relu"),     # Camada densa intermediária
    layers.Dropout(0.5),                      # Dropout para evitar overfitting
    layers.Dense(2, activation="softmax"),    # Saída com 2 classes (gato e cachorro)
])

# 5. Compilar o modelo
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Taxa de aprendizado reduzida
    loss="categorical_crossentropy",       # Função de perda para classificação multiclasse
    metrics=["accuracy"]                   # Métrica de desempenho
)

# 6. Treinar o modelo
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10  # Escolha um número maior para melhorar o desempenho
)

# 7. Avaliar o modelo
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {test_accuracy:.2f}")


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.7ETEXR_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 86s 126ms/step - accuracy: 0.9028 - loss: 2.3415 - val_accuracy: 0.9624 - val_loss: 0.3551
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 126s 117ms/step - accuracy: 0.9704 - loss: 0.2332 - val_accuracy: 0.9695 - val_loss: 0.2183
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 68s 116ms/step - accuracy: 0.9819 - loss: 0.0882 - val_accuracy: 0.9697 - val_loss: 0.1790
Epoch 4/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 82s 116ms/step - accuracy: 0.9870 - loss: 0.0544 - val_accuracy: 0.9716 - val_loss: 0.1715
Epoch 5/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 68s 117ms/step - accuracy: 0.9914 - loss: 0.0318 - val_accuracy: 0.9695 - val_loss: 0.1779
Epoch 6/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 72s 100ms/step - accuracy: 0.9921 - loss: 0.0283 - val_accuracy: 0.9703 - val_loss: 0.1839
Epoch 7/10
582/582 ━━━━━━━━━━